In [2]:
import kagglehub
path = kagglehub.dataset_download("lantian773030/pokemonclassification")


Resuming download from 266338304 bytes (171097367 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/lantian773030/pokemonclassification?dataset_version_number=1 (266338304/437435671) bytes left.


100%|██████████| 417M/417M [02:35<00:00, 1.10MB/s]

Extracting files...


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    path+'/PokemonData',
    target_size=(128, 128),   
    batch_size=16,            
    class_mode='categorical',
    subset='training'  
)

val_generator = train_datagen.flow_from_directory(
    path+'/PokemonData',
    target_size=(128, 128),   
    batch_size=16,
    class_mode='categorical',
    subset='validation' 
)
print("success")


C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Found 5511 images belonging to 150 classes.
Found 1309 images belonging to 150 classes.
success


In [39]:
import json

with open("class_labels.json", "w") as f:
    json.dump(train_generator.class_indices, f)


In [4]:
img_class=len(train_generator.class_indices)
print(img_class)

150


In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model


base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(128,128,3)
)


base_model.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x = Dense(64, activation="relu")(x)
output=Dense(img_class,activation="softmax")(x)

model=Model(inputs=base_model.input,outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

print("compiled")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 16s 2us/step
compiled


In [6]:
history=model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 181s 508ms/step - accuracy: 0.1595 - loss: 4.0819 - val_accuracy: 0.3598 - val_loss: 2.7269
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 40s 116ms/step - accuracy: 0.5523 - loss: 1.8684 - val_accuracy: 0.5661 - val_loss: 1.8033
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 41s 120ms/step - accuracy: 0.7362 - loss: 1.0660 - val_accuracy: 0.6066 - val_loss: 1.5564
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 42s 122ms/step - accuracy: 0.8352 - loss: 0.6821 - val_accuracy: 0.6494 - val_loss: 1.4191
Epoch 5/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 42s 122ms/step - accuracy: 0.8987 - loss: 0.4469 - val_accuracy: 0.6394 - val_loss: 1.4417
Epoch 6/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 43s 124ms/step - accuracy: 0.9428 - loss: 0.2896 - val_accuracy: 0.6746 - val_loss: 1.3601
Epoch 7/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 43s 124ms/step - accuracy: 0.9744 - loss: 0.1765 - val_accuracy: 0.6677 - val_loss: 1.3611
Epoch 8/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 43s 126ms/step - accuracy: 0.9851 - loss: 

In [38]:
model.save("pokemon_classifier.keras")

print("model saved")


model saved


In [34]:
import numpy as np
from tensorflow.keras.preprocessing import image
img_path = path+"/PokemonData/Cloyster/0049805978be4d23a3fa9e454928982c.jpg"
img=image.load_img(img_path,target_size=(128,128))

img=image.img_to_array(img)
img = img / 255.0     

img=np.expand_dims(img,axis=0)

result=model.predict(img)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


In [35]:
predicted_index = np.argmax(result[0])

class_labels = list(train_generator.class_indices.keys())
predicted_class = class_labels[predicted_index]

confidence = result[0][predicted_index] * 100

In [36]:
print(f"Predicted class: {predicted_class}, Confidence: {confidence:.2f}%")

preds = result[0]
class_labels = list(train_generator.class_indices.keys())

top3 = preds.argsort()[-3:][::-1]
for i in top3:
    print(f"{class_labels[i]}: {preds[i]*100:.2f}%")


Predicted class: Cloyster, Confidence: 97.35%
Cloyster: 97.35%
Seadra: 1.98%
Vaporeon: 0.36%
